In [1]:
import time
import bitsandbytes
import sys
import os
import torch
from transformers import AutoTokenizer, TextStreamer, GenerationConfig, AutoModelForCausalLM
sys.path.insert(0, '../..')
from drugs.dgenerate import DRUGS, print_top_k_logits_histogram
import drugs_conf as conf
current_script_dir = os.getcwd()
results_dir = os.path.join(current_script_dir, '..', 'results')

model_id = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id
streamer = TextStreamer(tokenizer)
sober_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True)
sober_model.eval()
torch.manual_seed(420)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/eron/miniconda3/envs/cuda_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/eron/miniconda3/envs/cuda_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


### Load sober content

In [2]:
input_ids = torch.load(os.path.join(results_dir, 'vergence', 'start_tensor.pt'))
print(tokenizer.decode(input_ids[0]))
with torch.no_grad():
    sober_outputs = sober_model(input_ids=input_ids, past_key_values=None, output_hidden_states=True)

<s> [INST]

Write an epic rap battle between William Rowe Hamilton and Lord Kelvin[/INST]

[Scene: A dark and dimly lit underground rap club, the air thick with anticipation. The crowd is on the edge of their seats as two of the greatest minds in science history take the stage. William Rowe Hamilton and Lord Kelvin are about to engage in an epic rap battle.]

William Rowe Hamilton:
Yo, I'm the king of the math game,
With equations that'll make your head spin like a flame,
I'm the one who brought you quaternions,
And I'll leave you in the dust, like a poor excuse for a reason.

Lord Kelvin:
Hold up, Hamilton, you ain't ready,
I'm the one who's got the science, the facts, and the


In [3]:
drugs = DRUGS()
def run_test(model, drug_type, base_name, sub_name, seed, dose_shape_callback=None, layer_begin=2, layer_end=31, dose_start=0, dose_end=3.14):
    with torch.no_grad():
        noise_stop_layer = layer_begin
        microdose= dose_start
        while noise_stop_layer < layer_end:
            dose_shape = dose_shape_callback(noise_stop_layer)
            drugs._set_typed_dose_shape(dose_shape[0], dose_shape[1], drug_type)
            while microdose < dose_end:
                torch.manual_seed(seed)
                drugs._set_typed_dose_theta(microdose, drug_type)
                result = model(input_ids=input_ids, past_key_values=None, output_hidden_states=True)
                clear_output(wait=True)
                print_top_k_logits_histogram(result.logits.to('cpu'), tokenizer, top_k=10)
                drugs.save_trace(base_name, sub_name, noise_stop_layer, result.logits, 10, result.hidden_states, tokenizer)            
                microdose += 0.05
            noise_stop_layer = noise_stop_layer + 1 #if noise_stop_layer < 4 else noise_stop_layer + 4
            microdose=dose_start

def layerwise_callback(at_layer):
    return([{'depth': (at_layer-1.1)/32, 'peakratio': 0},
            {'depth': (at_layer-1)/32, 'peakratio': 1},
            {'depth': (at_layer+1)/32, 'peakratio' : 1},
            {'depth': (at_layer+1.1)/32, 'peakratio' : 0}], 'ceil')

def fullstack_calback(at_layer):
    return([{'depth': -1, 'peakratio': 1},
            {'depth': (at_layer+1.1)/32, 'peakratio': 0}], 'ceil')

### Use DRµGS

In [4]:
from IPython.display import clear_output

model = drugs.inject(sober_model, tokenizer=tokenizer)
print_top_k_logits_histogram(sober_outputs.logits[:,:,:].to('cpu'), tokenizer=tokenizer)
microdose=0
noise_stop_layer = 2
seed = 420

run_test(model, 'A', f"Hamilton_A_dose", "single_layer",  seed, layerwise_callback)
drugs.set_A_dose_theta(0)
run_test(model, 'Q', f"Hamilton_Q_dose", "single_layer", seed, layerwise_callback)
drugs.set_Q_dose_theta(0)
run_test(model, 'K', f"Hamilton_K_dose", "single_layer", seed, layerwise_callback)
drugs.set_K_dose_theta(0)
run_test(model, 'V', f"Hamilton_V_dose", "single_layer", seed, layerwise_callback)
drugs.set_V_dose_theta(0)

--- 

#########                                                                                                      flow                    2.05%
##########                                                                                                  feature                    2.09%
##########                                                                                                     duty                    2.12%
############                                                                                                      c                    2.52%
############                                                                                                   cred                    2.56%
##################                                                                                                g                    3.75%
#####################                                                                                           cre                    4.49%
#######

In [5]:
drugs.set_V_dose_theta(0)
#run_test(model, 'A', "Hamilton_A_dose", "fullstack", seed, fullstack_calback)
drugs.set_A_dose_theta(0)
#run_test(model, 'Q', "Hamilton_Q_dose", "fullstack", seed, fullstack_calback)
drugs.set_Q_dose_theta(0)
run_test(model, 'K', "Hamilton_K_dose", "fullstack", seed, fullstack_calback)
drugs.set_K_dose_theta(0)
run_test(model, 'V', "Hamilton_V_dose", "fullstack", seed, fullstack_calback)
drugs.set_V_dose_theta(0)

--- 

##########################                                                                                Stein                    0.39%
############################                                                                               Stim                    0.41%
############################                                                                               vast                    0.42%
#############################                                                                               cup                    0.43%
#########################################                                                                   Chi                    0.60%
##############################################                                                            fluid                    0.68%
##############################################                                                               of                    0.68%
###################################